## Import libraries

In [0]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import tensorflow as tf
import pprint
import keras

## Data generator 

Keras does not have a generator that reads h5.
Pytorch handles h5 files.

In [0]:
import h5py
from keras.preprocessing.image import ImageDataGenerator

import sklearn.feature_extraction.image as ski

def create_patches(x1,params):
  
    x2 = np.zeros((params.batch_size,params.psize[0],params.psize[1],params.nch),dtype='float')
  
    x1 = x1[:,:params.window[0],:params.window[1],:]

    for i1 in np.arange(params.batch_size):
        x2[i1,:,:,0] = ski.extract_patches_2d(x1[i1,:,:,:],(params.psize[0],params.psize[1]),max_patches=params.npt)

        x2[i1,:,:,0] -= np.min(x2[i1,:,:,0])
        x2[i1,:,:,0] /= np.max(x2[i1,:,:,0])


    x2[:,:,:,1] = x2[:,:,:,0]
    x2[:,:,:,2] = x2[:,:,:,0]
    x2 -= .5
    x2 *= 2
    
    return x2


def generate_batches(files, idxt, params):
    nn = len(files)
    counter = 0
    cnt = 0
    
    datagen1 = ImageDataGenerator(brightness_range=[0.2,1.5],rotation_range=16)

    while True:

        hdf5_file1 = h5py.File(files[counter], "r")

        bs = int(params.batch_size/params.npt)

        np.random.shuffle(idxt)

        for nt in np.arange(params.ntrans):

            for cbatch in np.arange(0,idxt.shape[0],bs):
                idx = idxt[cbatch:(cbatch+bs)]
                idx.sort()

                x1 = (hdf5_file1["img"][idx,:,:,:])
                y1 = (hdf5_file1["labels"][idx,:2])



                if nt > 0:
                    it = datagen1.flow(x1, y1, batch_size=bs,shuffle=True)
                    x1,y1 = it.next()

                x11 = create_patches(x1,params)

                yield (x11,y1)

        hdf5_file1.close()

        counter += 1
        if counter == nn:
            counter = 0 
    return

Using TensorFlow backend.


## Deep CNN classifier using keras
Using a pre-trained ResNet50

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys, argparse
os.environ['KERAS_BACKEND'] = 'tensorflow'

import numpy as np

from keras.models import Model,load_model
from keras.layers import Dense,Input,Activation
from keras.regularizers import l1
from keras.utils import to_categorical

from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau,Callback

from keras.optimizers import RMSprop, SGD, Adam
from keras_applications.resnet_v2 import ResNet50V2

import keras.backend as K

In [0]:
from sklearn.model_selection import StratifiedKFold

def run_CNN(params):

    f = 0
    gen_train = generate_batches(files=[params.input+"train.hdf5"], idxt=np.arange(27430), params=params)
    gen_val = generate_batches(files=[params.input+"val.hdf5"], idxt=np.arange(5880),params=params)       


    model,model_name,loadmdl = MyModel(params,f)

    if loadmdl == 0:
            print('sgd = Adam(lr=3e-3)')
            sgd = Adam(3e-3)

            model = train_CNN(params,model,5,sgd,gen_train,gen_val,model_name)

    else:
            sgd = Adam(3e-5)

            model = train_CNN(params,model,5,sgd,gen_train,gen_val,model_name)

    return

In [0]:
def MyModel(params,f):

  
    model_name = params.models+str(f)+"epochs:002-val_acc:0.960.hdf5"
    if not os.path.isfile(model_name ):
        print("Creating Model : ", model_name)
        model_base = ResNet50V2(include_top=False, weights='imagenet', input_tensor=None, input_shape=(params.psize[0], params.psize[1],params.nch), pooling='avg', classes=3)

        x = model_base.get_layer('avg_pool').output
        dense = Dense(2,
                    kernel_initializer='he_normal',
                    bias_initializer='zeros',
                    kernel_regularizer=l1(0.01),
                    name='classifier')(x)
  
        prediction = Activation("softmax",name="softmax")(dense)

        model = Model(model_base.inputs, outputs=prediction, name='ResNet')

        loadmdl = 0
    else:
        print("Loading model: ", model_name)
        #model = load_model( model_name+'.model')
        #print("Creating Model : ", model_name)
        model_base = ResNet50V2(include_top=False, weights='imagenet', input_shape=(params.psize[0], params.psize[1],params.nch), pooling='avg',backend=keras.backend, layers=keras.layers, models=keras.models, utils=keras.utils)

        x = model_base.get_layer('avg_pool').output
        dense = Dense(2,
                    kernel_initializer='he_normal',
                    bias_initializer='zeros',
                    kernel_regularizer=l1(0.01),
                    name='classifier')(x)
  
        prediction = Activation("softmax",name="softmax")(dense)

        model = Model(model_base.inputs, outputs=prediction, name='ResNet')
      
        model.load_weights(model_name)


        loadmdl = 1


    return model,model_name,loadmdl


def train_CNN(params,model,nepochs,sgd,gen_train,gen_val,model_name):


        #print("Compiling model...",model_name)

        model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=["accuracy"])
        
        print("Compiled...") 
        
        model.summary()
        
        filepath=model_name+"epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"

 
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                  patience=3, min_lr=1e-8)

        earlyStopping = EarlyStopping(monitor='val_loss',
                           patience=10, 
                           verbose=1, mode='auto')

        checkpoint = ModelCheckpoint(filepath, monitor='val_loss',
                         verbose=0, save_best_only=True)

        nsamplesperfold = 1.*params.ntrans*params.npt/(params.batch_size)
        #print(nsamplesperfold)

        model.fit_generator(gen_train, steps_per_epoch = (27430*nsamplesperfold),
                  epochs=nepochs, verbose=1,
                  callbacks=[checkpoint,earlyStopping,reduce_lr],
                  validation_data=gen_val, validation_steps = (5880*nsamplesperfold))



        return model

In [0]:
sys.argv = ['foo']

parser = argparse.ArgumentParser()

parser.add_argument('-input',
                    type=str,
                    default='/home/ubuntu/volume1/BrainHack/data/BrainHack_quality_',
                    help='txt file containing input data filenames')

parser.add_argument('-models', type=str, 
                    default='/home/ubuntu/volume1/BrainHack/models/model_BrainHack_CTMR_h5_',
                    help='models prefix directory')

parser.add_argument('-psize', type=int, default=[128,128],
                    help='patch size extracted from dataset')

parser.add_argument('-window', type=int, default=[224,224],
                    help='area of interest ')

parser.add_argument('-img_size', type=int, default=[224,224],
                    help='area of interest ')

parser.add_argument('-nfolds', type=int, default=1,
                    help='number of folds for training')

parser.add_argument('-nch', type=int, default=3,
                    help='number of input channels')

parser.add_argument('-npt', type=int, default=1,
                    help='the number of patches from slices')

parser.add_argument('-ntrans', type=int, default=5,
                    help='number of transformations ')

parser.add_argument('-batch_size', type=int, default=10,
                    help='size of batch')

params = parser.parse_args()

run_CNN(params)

Loading model:  /content/drive/My Drive/model_BrainHack_CTMR_h5_0epochs:002-val_acc:0.960.hdf5
Compiled...
Model: "ResNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 66, 66, 64)   0           conv1_conv[0][0]    